In [37]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width=300)

numcores=8

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
library(jtools)

`%!in%` = Negate(`%in%`)

setDTthreads(numcores)

# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [2]:
#diagnosis codes
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=52850&ver=26 and https://www.aapc.com/codes/cpt-codes-range/93451-93533/10
angio_codes=c(93451,93452,93453,93454,93455,93456,93457,93458,93459,93460,93461,93462,93463,93464
              ,93503,93505,93530,93531,93532,93533)
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleid=57326&ver=13&keyword=electrocardiogram&keywordType=starts&areaId=all&docType=NCA,CAL,NCD,MEDCAC,TA,MCD,6,3,5,1,F,P&contractOption=all&sortBy=relevance&bc=1
ecg_codes=c(93000,93005,93010,93040,93041,93042)
#from https://scct.org/page/CardiacCTCodes include CTangio
cardiac_ct_codes=c(75571,75572,75573,75574)
#from https://cardiacmri.com/tech-guide/cpt-codes-relevant-to-cardiac-mri/
cardiac_mri_codes=c(75557,75559,75561,75563,75565)
# from https://medicarepaymentandreimbursement.com/2011/07/cardiovascular-stress-testing-cpt-93015.html and https://www.aapc.com/codes/cpt-codes-range/93000-93050/
stress_test_codes=c(93015,93016,93017,93018)
#from https://www.aapc.com/codes/cpt-codes-range/93303-93356/20     includes stress echo
echocardiography_codes=c(93303,93304,93306,93307,93308,93312,93313,93314,93315,93316,93317,93318
                         ,93320,93321,93325,93350,93351,93356,93352,93355,93356)
#from https://www.aapc.com/codes/cpt-codes-range/92920-92979/ and https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=57479#:~:text=CPT%20codes%2092928%2C%2092933%2C%2092929,are%20assigned%20to%20APC%200104.    includes balloon and stent
angioplasty_codes=c(92920,92921,92924,92925,92928,92929,92933,92934,92937,92938,92941,92943,92944
                    ,92973,92974,92975,92978,92979,93571,93572,"C9600","C9601","C9602","C9603"
                    ,"C9604","C9605","C9606","C9607","C9608")
#from https://www.medaxiom.com/clientuploads/webcast_handouts/Coding_for_CABG-Open_Heart_Procedures.pdf and https://www.aapc.com/codes/cpt-codes-range/33016-33999/10    did not include 33517-33530 since these are used in conjunction with 33533-33548 and not alone, did not include 33542,33545,33548 since these are also in conjunction )aneurismectomy and vsd resection
CABG_codes=c(33510,33511,33512,33513,33514,33516,33533,33534,33535,33536)
#from http://www.icd9data.com/2015/Volume1/390-459/430-438/default.htm and https://www.icd10data.com/ICD10CM/Codes/I00-I99/I60-I69/I63-
stroke_icd_9_codes=c(43301,43311,43321,43331,43381,43391,43401,43411,43491)
office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                     ,"99215")
IHD_icd_9_codes=c(410, 411, 412,413,414)
IHD_icd_10_codes=c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes=c(40,54,56,57,58,59,60,61,62,63,64,65,66,97,98,99)

primary_care_specialty_codes=c("01", "08", "11", "38")
surgery_specialty_codes=c("02","04","14","19","20","24","28","33","34","40","48","77","78","85","91")

#http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
#https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes=c("401","402","403","404","405")
hypertension_icd_10_codes=c("I10","I11","I12","I13","I15","I16")

#http://www.icd9data.com/2014/Volume1/460-519/490-496/default.htm
#https://www.icd10data.com/ICD10CM/Codes/J00-J99/J40-J47
copd_icd_9_codes=c("490","491","492","493","494","495","496")
copd_icd_10_codes=c("J40","J41","J42","J43","J44","J45","J47")

#http://www.icd9data.com/2015/Volume1/240-279/270-279/278/278.htm?__hstc=93424706.cdd51240e438a5219319ce13ccb23860.1648603374124.1648603374124.1648607295327.2&__hssc=93424706.9.1648607295327&__hsfp=908776442
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E65-E68/E66-
obesity_icd_9_codes=c("278")
obesity_icd_10_codes=c("E66")

#http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
#https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes=c("2962","2963")
depression_icd_10_codes=c("F32","F33")

#http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes=c("250")
diabetes_icd_10_codes=c("E08","E09","E10","E11","E13")

census_and_state_codes=read.csv("./results_mar/census_and_state_codes.csv")
census_and_state_codes[1:9,"state_code"] = paste("0",census_and_state_codes[1:9,"state_code"],sep="")

race_codes=data.frame(race_code = seq(0,6),
                      race = c("Unknown", "White", "Black", "Other", "Asian", "Hispanic", "North American Native")
                      )

sex_codes=data.frame(sex_code = seq(0,2),
                    sex = c("Unknown","Male","Female"))
                      
                      
                      

# Comparisons and analyses

## Comparisons between patients treated by integrated vs independent docotrs

I will compare the patients who were treated by the integrated vs non-integrated physicians, cardiologists and interventionists during the study period. Year 2013 did not divide cardiologists and inteventionists. So, We will not include this year in our comparisons of cardiologists and inteventionists.



## Reading the patient and physician integration results

In [3]:
yearly_calculations_stable_angina =
read_fst("results_apr/yearly_calculations_stable_angina_with_integration.fst"
         ,as.data.table = T) 
yearly_calculations_unstable_angina =
read_fst("results_apr/yearly_calculations_unstable_angina_with_integration.fst"
         ,as.data.table = T)
#physician_integration_stats = 
#read_fst("results_feb/physician_integration_stats.fst"
#        ,as.data.table = T)
#melted_physician_integration_stats=
#read_fst("results_feb/melted_physician_integration_stats.fst"
#        ,as.data.table = T)

In [4]:
tail(yearly_calculations_stable_angina,5)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,year_first_diagnosed,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLE

## Adding sex, race, and state names from codes

In [5]:
add_personal_details=function(data){
    
  require(tidyverse)
  require(dtplyr)
  require(lubridate)
  
  result=data %>%
  left_join(.,census_and_state_codes[,-1],by=c("state_code_at_diagnosis"="state_code"))%>%
  left_join(.,race_codes,by=c("race_code_at_diagnosis"="race_code"))%>%
  left_join(.,sex_codes,by=c("sex_code_at_diagnosis"="sex_code"))%>%
  mutate(
    age_group=case_when(
      age_at_diagnosis<75 & age_at_diagnosis>64 ~ "65-74",
      age_at_diagnosis>74 & age_at_diagnosis<85 ~ "75-84",
      age_at_diagnosis>84 ~ "85+"
    )
  )%>%
  as.data.table()
  
  return(result)
}

In [6]:
yearly_calculations_stable_angina=add_personal_details(yearly_calculations_stable_angina)
yearly_calculations_unstable_angina=add_personal_details(yearly_calculations_unstable_angina)

Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:reshape’:

    stamp


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week, yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## Filtering the data for the IV model

In [7]:
data_for_modelling_filter=function(data){
  library(tidyverse)
  library(dtplyr)
  
  data%>%
  filter(state_code_at_diagnosis %!in% non_us_state_codes &
         age_at_diagnosis>=65&
         ((died_in_one_year_after_diagnosis==T & HMO_INDICATOR_sum>0) |
          (died_in_one_year_after_diagnosis==F & HMO_INDICATOR_sum==12))&
         ((died_in_one_year_after_diagnosis==T & ENTITLEMENT_BUY_IN_IND_sum>0) |
          (died_in_one_year_after_diagnosis==F & ENTITLEMENT_BUY_IN_IND_sum==12))&
         year_first_diagnosed!=2013&
         !is.na(most_common_cardiologist_PRF_PHYSN_NPI)
        )%>%
  as.data.table()
}

In [8]:
yearly_calculations_stable_angina=data_for_modelling_filter(yearly_calculations_stable_angina)
yearly_calculations_unstable_angina=data_for_modelling_filter(yearly_calculations_unstable_angina)

In [9]:
colnames(yearly_calculations_stable_angina)

[1] "DESY_SORT_KEY"                                                                   "first_diagnosis"                                                                 "tot_allowed_carrier"                                                            
  [4] "office_visit_count"                                                              "office_visit_cost_carrier"                                                       "cardiology_visit_count"                                                         
  [7] "distinct_clinicians"                                                             "distinct_cardiologists"                                                          "distinct_primary_care_physicians"                                               
 [10] "distinct_surgeons"                                                               "distinct_other_specialties"                                                      "catheterization_count"                                                          
 [13] "catheterization_cost_carrier"                                                    "ecg_count"                                                                       "ecg_cost_carrier"                                                               
 [16] "cardiac_ct_count"                                                                "cardiac_ct_cost_carrier"                                                         "cardiac_mri_count"                                                              
 [19] "cardiac_mri_cost_carrier"                                                        "stress_test_count"                                                               "stress_test_cost_carrier"                                                       
 [22] "echocardiography_count"                                                          "echocardiography_cost_carrier"                                                   "angioplasty_count"                                                              
 [25] "angioplasty_cost_carrier"                                                        "CABG_count"                                                                      "CABG_cost_carrier"                                                              
 [28] "stable_angina"                                                                   "unstable_angina"                                                                 "MI"                                                                             
 [31] "cardiac_arrest"                                                                  "stroke"                                                                          "hypertension"                                                                   
 [34] "copd"                                                                            "obesity"                                                                         "depression"                                                                     
 [37] "diabetes"                                                                        "icd_9_pure"                                                                      "icd_10_pure"                                                                    
 [40] "year_first_diagnosed"                                                            "tot_allowed_carrier_cardiology_related"                                          "office_visit_count_cardiology_related"                                          
 [43] "office_visit_cost_carrier_cardiology_related"                                    "cardiology_visit_count_cardiology_related"                                       "distinct_clinicians_cardiology_related"                                         
 [46] "distinct_cardiologists_cardiology_related"                                       "distinct_primary_care_physicians_cardiology_related"                             "distinct_surgeons_cardiology_related"              

### Number of patients in each of stable and unstable angina groups after the filters

In [13]:
length(unique(yearly_calculations_stable_angina$DESY_SORT_KEY))
length(unique(yearly_calculations_unstable_angina$DESY_SORT_KEY))

[1] 15316

[1] 9691

In [123]:
sum(yearly_calculations_stable_angina[,primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS]<0.5,na.rm=T)
sum(yearly_calculations_unstable_angina[,primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS]<0.5,na.rm=T)

[1] 9591

[1] 6067

In [112]:
sum(yearly_calculations_stable_angina[,is.na(primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS)],na.rm=T)
sum(yearly_calculations_unstable_angina[,is.na(primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS)],na.rm=T)

[1] 4897

[1] 3111

In [121]:
sum(yearly_calculations_stable_angina[,cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS]>0.5,na.rm=T)
sum(yearly_calculations_unstable_angina[,cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS]>0.5,na.rm=T)

[1] 1046

[1] 626

In [116]:
sum(yearly_calculations_stable_angina[,is.na(primary_care_physician_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS)],na.rm=T)
sum(yearly_calculations_unstable_angina[,is.na(primary_care_physician_2013_avg_in_year_before_diagnosis_non_exclusive_HCPCS)],na.rm=T)

[1] 5638

[1] 3527

## Naive models without VI

### Stable Angina

In [315]:
naive_model_stable_angina=glm(catheterization_count>0~
               (cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS<0.75):
               (primary_care_physician_avg_in_year_of_diagnosis_non_exclusive_HCPCS<0.75)+             
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_stable_angina,family="binomial"
              )



In [319]:
summary_naive_model_stable_angina=summ(naive_model_stable_angina)

In [320]:
data.frame(exp(summary_naive_model_stable_angina$coeftable))

,Est.,S.E.,z.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),10.6231226,1.331015,3.881766e+03,1.000000
sexMale,1.3231315,1.043046,7.679641e+02,1.000000
raceBlack,1.3953780,1.201527,6.139210e+00,1.072048
raceHispanic,1.2336422,1.281310,2.332771e+00,1.487302
raceNorth American Native,1.7028752,1.429737,4.432842e+00,1.146228
raceOther,1.4021714,1.259025,4.338399e+00,1.152852
raceUnknown,1.5358810,1.281776,5.632426e+00,1.087511
raceWhite,1.8339083,1.178807,3.990735e+01,1.000227
age_at_diagnosis,0.9546065,1.003045,2.307772e-07,1.000000


### Unstable Angina

In [43]:
naive_model_unstable_angina=lm(catheterization_count>0~
               (cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>0.5)+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_unstable_angina
              )

In [44]:
summary_naive_model_unstable_angina=summ(naive_model_unstable_angina)

In [45]:
data.frame(summary_naive_model_unstable_angina$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.336713948,0.0745291861,17.9354427,1.665390e-70
cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS > 0.5TRUE,0.025011833,0.0131431251,1.9030355,5.707312e-02
sexMale,0.040978236,0.0112214115,3.6517898,2.621443e-04
raceBlack,-0.007198215,0.0476561850,-0.1510447,8.799444e-01
raceHispanic,-0.007700903,0.0629013213,-0.1224283,9.025630e-01
raceNorth American Native,-0.043043487,0.0990011479,-0.4347777,6.637360e-01
raceOther,0.014709273,0.0650251093,0.2262091,8.210448e-01
raceUnknown,-0.039906407,0.0653590675,-0.6105719,5.415011e-01
raceWhite,0.093903243,0.0429482953,2.1864254,2.881445e-02


## The IV model

### Stable Angina

#### Stage 1

In [277]:
yearly_calculations_stable_angina[,primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half
                                  :=(primary_care_physician_2013_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.75)
                                 ]

In [278]:
iv_model_stable_angina_stage_1=lm((cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.75)~
               primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75,]
              )

In [279]:
summary_iv_model_stable_angina_stage_1=summ(iv_model_stable_angina_stage_1,robust = "HC3")

In [280]:
data.frame(summary_iv_model_stable_angina_stage_1$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.333642e-01,0.0472341449,2.8234706,4.764462e-03
primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_halfTRUE,2.347626e-01,0.0331631500,7.0790209,1.596956e-12
sexMale,1.093114e-02,0.0072288930,1.5121453,1.305433e-01
raceBlack,1.829838e-02,0.0237635112,0.7700201,4.413146e-01
raceHispanic,-1.419492e-02,0.0286058222,-0.4962248,6.197518e-01
raceNorth American Native,1.977317e-01,0.1045102908,1.8919833,5.853556e-02
raceOther,2.097694e-02,0.0319262533,0.6570436,5.111751e-01
raceUnknown,6.457575e-02,0.0563863437,1.1452374,2.521511e-01
raceWhite,2.256718e-02,0.0198186446,1.1386841,2.548749e-01


#### Predictions

In [281]:
predictions_iv_model_stable_angina_stage_1=predict(iv_model_stable_angina_stage_1,
        newdata = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75,
                                                    .(primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half,
                                                    sex,
                                                    race,
                                                    age_at_diagnosis,
                                                    census_region,
                                                    hypertension,
                                                    copd,
                                                    obesity,
                                                    depression,
                                                    diabetes,
                                                    wscore)])

In [282]:
head(predictions_iv_model_stable_angina_stage_1)

1          2          3          4          5          6 
0.07046924 0.10836655 0.09665790 0.09499387 0.07278398         NA

In [283]:
yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75,predictions_iv:=predictions_iv_model_stable_angina_stage_1]



#### Stage 2

##### Catheterization

In [284]:
iv_model_stable_angina_stage_2_catheterization=lm(catheterization_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [285]:
summary_iv_model_stable_angina_stage_2_catheterization=summ(iv_model_stable_angina_stage_2_catheterization,robust = "HC3")

In [286]:
data.frame(summary_iv_model_stable_angina_stage_2_catheterization$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.1597097505,0.075803968,15.29880007,5.646207e-52
predictions_iv,0.3716444419,0.150380621,2.47135860,1.348433e-02
sexMale,0.0477444883,0.011731380,4.06981013,4.758301e-05
raceBlack,0.0255886287,0.041302573,0.61954079,5.355809e-01
raceHispanic,0.0848496483,0.058440581,1.45189604,1.465765e-01
raceNorth American Native,0.0747991228,0.117893659,0.63446264,5.258002e-01
raceOther,0.0081734940,0.053196356,0.15364763,8.778921e-01
raceUnknown,0.0271378634,0.079101064,0.34307836,7.315501e-01
raceWhite,0.0945457851,0.034911755,2.70813613,6.783111e-03


##### Echo

In [287]:
iv_model_stable_angina_stage_2_echocardiography=lm(echocardiography_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [288]:
summary_iv_model_stable_angina_stage_2_echocardiography=summ(iv_model_stable_angina_stage_2_echocardiography,robust = "HC3")

In [289]:
data.frame(summary_iv_model_stable_angina_stage_2_echocardiography$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.4738910443,0.0821000341,5.7721175,8.171977e-09
predictions_iv,-0.0736802152,0.1504405579,-0.4897630,6.243174e-01
sexMale,-0.0054146623,0.0119855041,-0.4517676,6.514508e-01
raceBlack,-0.0367197943,0.0455542020,-0.8060682,4.202316e-01
raceHispanic,-0.1033878631,0.0609119864,-1.6973320,8.967956e-02
raceNorth American Native,0.0230681232,0.1122539640,0.2054994,8.371881e-01
raceOther,-0.0196114887,0.0585795955,-0.3347836,7.377986e-01
raceUnknown,-0.0226476710,0.0807363948,-0.2805138,7.790919e-01
raceWhite,-0.1070386116,0.0398119407,-2.6886057,7.192557e-03


##### Stress test

In [290]:
iv_model_stable_angina_stage_2_stress_test=lm(stress_test_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [291]:
summary_iv_model_stable_angina_stage_2_stress_test=summ(iv_model_stable_angina_stage_2_stress_test,robust = "HC3")

In [292]:
data.frame(summary_iv_model_stable_angina_stage_2_stress_test$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.6532589105,0.0775927640,8.41906998,4.580297e-17
predictions_iv,0.0061864135,0.1401729922,0.04413413,9.647988e-01
sexMale,0.0085616560,0.0114503577,0.74771952,4.546551e-01
raceBlack,0.0342439790,0.0459533067,0.74519075,4.561821e-01
raceHispanic,0.0288663133,0.0617630155,0.46737215,6.402486e-01
raceNorth American Native,0.0759079665,0.1252879823,0.60586790,5.446226e-01
raceOther,0.0779837088,0.0596476041,1.30740723,1.911185e-01
raceUnknown,0.0646470801,0.0836772688,0.77257636,4.397999e-01
raceWhite,0.0338474672,0.0401313303,0.84341752,3.990246e-01


##### CABG

In [293]:
iv_model_stable_angina_stage_2_CABG=lm(CABG_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [294]:
summary_iv_model_stable_angina_stage_2_CABG=summ(iv_model_stable_angina_stage_2_CABG,robust = "HC3")

In [295]:
data.frame(summary_iv_model_stable_angina_stage_2_CABG$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.185928337,0.0295864839,6.2842323,3.497930e-10
predictions_iv,0.034327559,0.0685584604,0.5007049,6.165950e-01
sexMale,0.041772331,0.0051427487,8.1225690,5.374314e-16
raceBlack,-0.001469411,0.0112661236,-0.1304274,8.962321e-01
raceHispanic,0.010077333,0.0171772424,0.5866677,5.574463e-01
raceNorth American Native,0.083518231,0.0716070309,1.1663412,2.435173e-01
raceOther,0.017618094,0.0185965989,0.9473826,3.434774e-01
raceUnknown,-0.017804966,0.0255779259,-0.6961067,4.863857e-01
raceWhite,0.029042270,0.0089242293,3.2543168,1.142198e-03


##### Angioplasty

In [296]:
iv_model_stable_angina_stage_2_angioplasty=lm(angioplasty_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.75]
                   )

In [297]:
summary_iv_model_stable_angina_stage_2_angioplasty=summ(iv_model_stable_angina_stage_2_angioplasty,robust = "HC3")

In [298]:
data.frame(summary_iv_model_stable_angina_stage_2_angioplasty$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.320062733,0.0551957867,5.7986805,6.982839e-09
predictions_iv,0.237291807,0.1224641563,1.9376429,5.270805e-02
sexMale,0.053064623,0.0086312783,6.1479449,8.292586e-10
raceBlack,-0.022968800,0.0290875343,-0.7896441,4.297631e-01
raceHispanic,0.055443240,0.0451132430,1.2289793,2.191220e-01
raceNorth American Native,0.050780051,0.1028496497,0.4937309,6.215121e-01
raceOther,0.020641718,0.0402208700,0.5132091,6.078217e-01
raceUnknown,0.037526992,0.0642249848,0.5843052,5.590343e-01
raceWhite,0.022157732,0.0251264549,0.8818487,3.778897e-01


### Unstable Angina

#### Stage 1

In [150]:
yearly_calculations_unstable_angina[,primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half
                                  :=(primary_care_physician_2013_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.6)
                                 ]

In [151]:
iv_model_unstable_angina_stage_1=lm((cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.6)~
               primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2,]
              )

In [152]:
summary_iv_model_unstable_angina_stage_1=summ(iv_model_unstable_angina_stage_1,robust = "HC3")

In [153]:
data.frame(summary_iv_model_unstable_angina_stage_1$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.170199142,0.0731688368,2.32611518,2.006093e-02
primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_halfTRUE,0.228572083,0.0467151455,4.89289031,1.031820e-06
sexMale,0.026997849,0.0112135256,2.40761468,1.610103e-02
raceBlack,0.020060818,0.0381112343,0.52637544,5.986559e-01
raceHispanic,-0.030708970,0.0443526348,-0.69238209,4.887366e-01
raceNorth American Native,0.144051039,0.1352543226,1.06503834,2.869213e-01
raceOther,0.002866352,0.0490503462,0.05843693,9.534034e-01
raceUnknown,-0.007221504,0.0724412873,-0.09968768,9.205972e-01
raceWhite,0.046220281,0.0326522949,1.41552933,1.569893e-01


#### Predictions

In [154]:
predictions_iv_model_unstable_angina_stage_1=predict(iv_model_unstable_angina_stage_1,
        newdata = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2,
                                                    .(primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half,
                                                    sex,
                                                    race,
                                                    age_at_diagnosis,
                                                    census_region,
                                                    hypertension,
                                                    copd,
                                                    obesity,
                                                    depression,
                                                    diabetes,
                                                    wscore)])

In [155]:
head(predictions_iv_model_unstable_angina_stage_1)

1          2          3          4          5          6 
0.13858338 0.13716212 0.08239007 0.13973876         NA 0.05930227

In [156]:
yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2,predictions_iv:=predictions_iv_model_unstable_angina_stage_1]

#### Stage 2

##### Catheterization

In [157]:
iv_model_unstable_angina_stage_2_catheterization=lm(catheterization_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<=0.2]
                   )

In [158]:
summary_iv_model_unstable_angina_stage_2_catheterization=summ(iv_model_unstable_angina_stage_2_catheterization,robust = "HC3")

In [159]:
data.frame(summary_iv_model_unstable_angina_stage_2_catheterization$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.591985057,0.114545796,13.8982408,6.028080e-43
predictions_iv,0.228767518,0.217634004,1.0511571,2.932484e-01
sexMale,0.025352219,0.016344700,1.5510972,1.209556e-01
raceBlack,-0.147255646,0.069193022,-2.1281864,3.338110e-02
raceHispanic,-0.249757467,0.091098833,-2.7416100,6.140406e-03
raceNorth American Native,-0.240245343,0.154674262,-1.5532341,1.204444e-01
raceOther,-0.126814353,0.091728502,-1.3824967,1.668945e-01
raceUnknown,-0.225130669,0.115669674,-1.9463241,5.168403e-02
raceWhite,-0.056089367,0.063290258,-0.8862243,3.755486e-01


##### Echo

In [100]:
iv_model_unstable_angina_stage_2_echocardiography=lm(echocardiography_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [101]:
summary_iv_model_unstable_angina_stage_2_echocardiography=summ(iv_model_unstable_angina_stage_2_echocardiography,robust = "HC3")

In [102]:
data.frame(summary_iv_model_unstable_angina_stage_2_echocardiography$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.2535125379,0.116332316,2.17920992,2.937145e-02
predictions_iv,-0.1917779739,0.196810555,-0.97442931,3.298995e-01
sexMale,0.0186825457,0.014854038,1.25774189,2.085550e-01
raceBlack,-0.0261798241,0.065244102,-0.40125963,6.882494e-01
raceHispanic,0.0348960360,0.084801138,0.41150434,6.807238e-01
raceNorth American Native,0.0239281752,0.162055240,0.14765444,8.826226e-01
raceOther,0.0242055893,0.081735011,0.29614713,7.671324e-01
raceUnknown,0.0640591655,0.108728559,0.58916596,5.557816e-01
raceWhite,-0.0008222624,0.059552134,-0.01380744,9.889843e-01


##### CABG

In [103]:
iv_model_unstable_angina_stage_2_CABG=lm(CABG_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [104]:
summary_iv_model_unstable_angina_stage_2_CABG=summ(iv_model_unstable_angina_stage_2_CABG,robust = "HC3")

In [105]:
data.frame(summary_iv_model_unstable_angina_stage_2_CABG$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.375835931,0.0537227543,6.9958426,3.054795e-12
predictions_iv,-0.180735917,0.0770558261,-2.3455192,1.904687e-02
sexMale,0.049835240,0.0080415745,6.1971993,6.299932e-10
raceBlack,0.004795382,0.0285594967,0.1679085,8.666634e-01
raceHispanic,-0.028427136,0.0357253268,-0.7957138,4.262433e-01
raceNorth American Native,0.138752358,0.1161342677,1.1947581,2.322490e-01
raceOther,0.020060964,0.0415788292,0.4824803,6.294900e-01
raceUnknown,-0.065034452,0.0479793323,-1.3554681,1.753413e-01
raceWhite,0.031565942,0.0253297272,1.2462014,2.127599e-01


##### Angioplasty

In [109]:
iv_model_unstable_angina_stage_2_angioplasty=lm(angioplasty_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_unstable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [110]:
summary_iv_model_unstable_angina_stage_2_angioplasty=summ(iv_model_unstable_angina_stage_2_angioplasty,robust = "HC3")

In [111]:
data.frame(summary_iv_model_unstable_angina_stage_2_angioplasty$coeftable)

,Est.,S.E.,t.val.,p
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.3694929284,0.1031784351,3.58110615,3.460153e-04
predictions_iv,0.1118077616,0.1756437978,0.63655969,5.244464e-01
sexMale,0.0760251704,0.0130761593,5.81402906,6.551324e-09
raceBlack,-0.1727680646,0.0607122867,-2.84568534,4.453061e-03
raceHispanic,-0.1316956000,0.0758641589,-1.73593963,8.264783e-02
raceNorth American Native,-0.1254717592,0.1377896464,-0.91060368,3.625565e-01
raceOther,0.0050744317,0.0834263852,0.06082526,9.515013e-01
raceUnknown,-0.1275061379,0.0982838616,-1.29732528,1.945905e-01
raceWhite,-0.0614355609,0.0582006503,-1.05558203,2.912199e-01
